In [1]:
import numpy as np
import pandas as pd
import pyddm
import os
import models
import utils

In [2]:
def get_model_measures(model, condition):
    sol = model.solve(condition)
    mean_rt_go = np.sum(sol.pdf(choice="Go")*model.t_domain())*model.dt / sol.prob(choice="Go")
    mean_rt_stay = np.sum(sol.pdf(choice="Stay")*model.t_domain())*model.dt / sol.prob(choice="Stay")
    
    return condition["tta_0"], condition["d_0"], condition["a_values"], condition["a_duration"],\
           sol.prob(choice="Go"), mean_rt_go, mean_rt_stay

def get_model_rt_distr(model, condition, kind="cdf"):
    sol = model.solve(condition)
    return pd.DataFrame({"tta_0": condition["tta_0"],
                         "d_0": condition["d_0"],
                         "a_values": condition["a_values"],
                         "a_duration": condition["a_duration"],
                         "t": model.t_domain(),
                         "rt_corr_distr": (sol.cdf(choice="Go") if kind=="cdf" else sol.pdf(choice="Go"))/sol.prob(choice="Go"),
                         "rt_error_distr": (sol.cdf(choice="Stay") if kind=="cdf" else sol.pdf(choice="Stay"))/sol.prob(choice="Stay")})

def initialize_model(model_no, param_set, state_interpolators):
    overlay = models.OverlayNonDecisionGaussian(ndt_location=param_set.ndt_location, ndt_scale=param_set.ndt_scale)   
    IC = pyddm.ICPointRatio(x0=param_set.x0)
    
    if model_no==1:
        IC = pyddm.ICPointRatio(x0=0)
        drift = models.DriftAccelerationDependent(alpha=param_set.alpha, beta_d=param_set.beta_d, beta_a=param_set.beta_a, theta=param_set.theta, state_interpolators=state_interpolators)
        bound = models.BoundCollapsingTta(b_0=param_set.b_0, k=param_set.k, tta_crit=param_set.tta_crit, state_interpolators=state_interpolators)
    if model_no==2:
        drift = models.DriftAccelerationDependent(alpha=param_set.alpha, beta_d=param_set.beta_d, beta_a=param_set.beta_a, theta=param_set.theta, state_interpolators=state_interpolators)
        bound = models.BoundCollapsingTta(b_0=param_set.b_0, k=param_set.k, tta_crit=param_set.tta_crit, state_interpolators=state_interpolators)
    elif model_no==3:
        drift = models.DriftAccelerationDependent(alpha=param_set.alpha, beta_d=param_set.beta_d, beta_a=param_set.beta_a, theta=param_set.theta, state_interpolators=state_interpolators)
        bound = models.BoundCollapsingGeneralizedGap(b_0=param_set.b_0, k=param_set.k, alpha=param_set.alpha, beta_d=param_set.beta_d, 
                                                     beta_a=param_set.beta_a, theta=param_set.theta, state_interpolators=state_interpolators)
    elif model_no==4:
        bound = pyddm.BoundConstant(B=param_set.B)
        drift = models.DriftAccelerationDependent(alpha=param_set.alpha, beta_d=param_set.beta_d, beta_a=0, theta=param_set.theta, state_interpolators=state_interpolators)
    elif model_no==5:
        bound = pyddm.BoundConstant(B=param_set.B)
        drift = models.DriftAccelerationDependent(alpha=param_set.alpha, beta_d=param_set.beta_d, beta_a=param_set.beta_a, theta=param_set.theta, state_interpolators=state_interpolators)
        
    model = pyddm.Model(name="Model %i" % model_no, drift=drift, bound=bound, overlay=overlay, IC=IC,
                      noise=pyddm.NoiseConstant(noise=1), T_dur=models.ModelAccelerationDependent.T_dur, choice_names=("Go", "Stay"))        
    return model

def simulate_model(model_no, param_set, conditions, ret="measures"):
    """
    Set ret to "measures" or "rt_cdf" or "rt_pdf" for saving p_turn and mean RT or RT CDF or RT PDF
    """
    state_interpolators = utils.get_state_interpolators(conditions=conditions)
    model = initialize_model(model_no, param_set, state_interpolators)
    if ret=="measures":
        sim_result = pd.DataFrame([get_model_measures(model, condition) for condition in conditions],
                                  columns=["tta_0", "d_0", "a_values", "a_duration", "is_gap_accepted", "RT_go", "RT_stay"])
    else:
        sim_result = pd.concat([get_model_rt_distr(model, condition, kind=ret[-3:]) for condition in conditions])
    sim_result["subj_id"] = param_set.subj_id
    return sim_result

In [3]:
def save_sim_results(loss, model_no, parameters=None, conditions=None, ret="measures", vincent=False):
    file_name="subj_all_parameters_fitted.csv"
    path = os.path.join("modeling/fit_results_%s" % (loss) + ("_vincent" if vincent else ""), "model_%i" % (model_no))

    if parameters is None:
        parameters = pd.read_csv(os.path.join(path, file_name))

    sim_results = [simulate_model(model_no, param_set, conditions, ret=ret)
                   for idx, param_set in parameters.iterrows()]

    sim_results = pd.concat(sim_results)
    sim_results.to_csv(os.path.join(path, file_name.replace("parameters_fitted", "sim_" + ret)), index=False)

In [4]:
conditions = [{"tta_0": tta_0, "d_0": d_0, "a_values": a_values, "a_duration": a_duration}
              for tta_0 in [4.5, 5.5]
              for d_0 in [80]
              for a_values in [(0., 0., 0., 0.),
                               (0., 4, 4, 0.),
                               (0., 4, -4, 0.),
                               (0., -4, 4, 0.),
                               (0., -4, -4, 0.)]
              for a_duration in [1.0]]

# Saving model-predicted measures

In [16]:
save_sim_results(loss="bic", model_no=2, conditions=conditions, ret="measures")

Model(name='Model 2',
      drift=DriftAccelerationDependent(alpha=0.251482960156058, beta_d=0.0060209623152879, beta_a=0.195575398082358, theta=8.123555358381736, state_interpolators={"{'tta_0': 4.5, 'd_0': 80, 'a_values': (0.0, 0.0, 0.0, 0.0), 'a_duration': 1.0}": (<scipy.interpolate._interpolate.interp1d object at 0x00000128EF1214F0>, <scipy.interpolate._interpolate.interp1d object at 0x0000012891704E50>, <scipy.interpolate._interpolate.interp1d object at 0x000001289189C220>), "{'tta_0': 4.5, 'd_0': 80, 'a_values': (0.0, 4, 4, 0.0), 'a_duration': 1.0}": (<scipy.interpolate._interpolate.interp1d object at 0x00000128918A70E0>, <scipy.interpolate._interpolate.interp1d object at 0x00000128918A72C0>, <scipy.interpolate._interpolate.interp1d object at 0x00000128918A75E0>), "{'tta_0': 4.5, 'd_0': 80, 'a_values': (0.0, 4, -4, 0.0), 'a_duration': 1.0}": (<scipy.interpolate._interpolate.interp1d object at 0x00000128918A7360>, <scipy.interpolate._interpolate.interp1d object at 0x00000128918A73

In [28]:
save_sim_results(loss="bic", model_no=4, conditions=conditions, ret="measures")

In [5]:
save_sim_results(loss="bic", model_no=5, conditions=conditions, ret="measures")

# Testing the effects of $\beta_a$

In [23]:
path = "modeling/fit_results_excluded_nan_rt/model_acceleration_dependent_cross_validation"
parameters = pd.read_csv(os.path.join(path, "subj_all_parameters_fitted.csv"))
parameters["beta_a"] = 0

save_sim_results(model_no=3, file_name="subj_all_parameters_fitted.csv", parameters=parameters, conditions=conditions, cross_validation=True, ret="measures", prefix="beta_a_0_")

In [9]:
path = "modeling/fit_results_excluded_nan_rt/model_acceleration_dependent_cross_validation"
parameters = pd.read_csv(os.path.join(path, "subj_all_parameters_fitted.csv"))
parameters["beta_a"] = 1

save_sim_results(model_no=3, file_name="subj_all_parameters_fitted.csv", parameters=parameters, conditions=conditions, cross_validation=True, ret="measures", prefix="beta_a_1_")

Model(name='Model 3',
      drift=DriftAccelerationDependent(alpha=0.3820280887987444, beta_d=0.1697230601511917, beta_a=1, theta=16.18984170700848),
      noise=NoiseConstant(noise=1),
      bound=BoundCollapsingTta(b_0=2.5297150095538457, k=0.0075507906549354, tta_crit=3.2063611626853987),
      IC=ICPointSourceCenter(),
      overlay=OverlayNonDecisionGaussian(ndt_location=0.0017032654042604, ndt_scale=0.2710190910102154),
      dx=0.005,
      dt=0.005,
      T_dur=6.0,
  choice_names=('Go', 'Stay'))
{'tta_0': 4.5, 'd_0': 80, 'a_values': (0.0, 0.0, 0.0, 0.0), 'a_duration': 1.0}
0.2653471376358351
0.7157150914346334
{'tta_0': 4.5, 'd_0': 80, 'a_values': (0.0, 4, 4, 0.0), 'a_duration': 1.0}
0.06982659446730552
0.9102956961933933
{'tta_0': 4.5, 'd_0': 80, 'a_values': (0.0, 4, -4, 0.0), 'a_duration': 1.0}
0.07524119270576171
0.904881033912379
{'tta_0': 4.5, 'd_0': 80, 'a_values': (0.0, -4, 4, 0.0), 'a_duration': 1.0}
0.717078740663487
0.2563848137707281
{'tta_0': 4.5, 'd_0': 80, 'a_val

# Predicting the effect of other nudges

In [10]:
a_durations = np.linspace(0.1, 3.0, 11)
a_magnitudes = np.linspace(0.0, 5.0, 11)

conditions = [{"tta_0": 5, "d_0": 90, "a_values": (0.0, -a_magnitude, a_magnitude, 0.0), "a_duration": a_duration}
              for a_duration in a_durations
              for a_magnitude in a_magnitudes]

In [11]:
save_sim_results(model_no=3, file_name="subj_all_parameters_fitted.csv", conditions=conditions, cross_validation=True,
                 ret="measures", prefix="prediction_")